In [5]:
# libraries
import time
import ssl
import imaplib
import smtplib
import email

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Sending Mail
from email.utils import formataddr
from email.mime.text import MIMEText
# from email.MIMEMultipart import MIMEMultipart
from email.mime.multipart import MIMEMultipart
# from email.mime.MIMEBase import MIMEBase
# from email import Encoders
import tqdm
import os
import pandas as pd

In [6]:
## Initializations:

mail_user = ''
mail_pass = ''

mail_host = 'smtp.hostinger.com'
save_host = 'imap.hostinger.com'
save_host = 993

In [4]:
prompt = """You are a hyper-personalized marketing app. You need to create enganing content i.e an email explaing the services of our organization i.e \
BB AI App, to out client company. Below is the client company website content summarized. 
#### Client website content: 
{client_content}

Now, generate an email so that client accepts our services.
"""

In [5]:
subject = ""

In [8]:
template = """
Dear Sir/Madam,
[Company Name]

I hope this message finds you well. We are Dr Macs Bio Pharma Pvt Ltd, a trusted pharmaceutical company specializing in high-quality APIs and intermediates.

Our services involve:
    1. Providing high quality APIs and Intermediates
        We have attached our brochure where you can find our list of products. 
        
    2. Technical Support
        Drug Discovery ( CRO )
        Route Scouting
        Contract Manufacturing ( CMO  )
        
    3. We are actively looking out for any collaboration to enhance sourcing of any materials or products from your organization, As we are
       very well known and trusted vendor for almost a 100 companies in India.
       
I am reaching out to inquire about the possibility of establishing a mutually beneficial partnership with your esteemed organization. We are interested in sourcing specific raw materials and API intermediates to support our manufacturing processes.

Key Points of Interest:
    Quality Assurance
    Capacity and Timeliness
    Regulatory Compliance

We are looking to discuss for possibility of a long-term collaboration. Additionally, we would appreciate receiving your product catalog, price list, and any relevant certifications.
Please feel free to reach out to me at yesh@drmacspharma.com | +91 8143090222, to discuss this further or to schedule a meeting at your earliest convenience. Please find our company brochure for additional information.

Thank you for considering our inquiry, and we look forward to the prospect of working together.

Best regards,

Yeshwanth
Director
Dr Macs Bio Pharma Pvt Ltd
Email: yesh@drmacspharma.com
Mobile: +91 8143090222
Website: www.drmacspharma.com

Address:
Plot No: 79/ B&C, EPIP, Pashamylaram (V), 
Patancheru (M) Sangareddy, TELANGANA, INDIA.
Pincode: 502307

IMPORTANT: This email may discuss privileged and confidential information. If you are not the intended recipient, you are required to inform the sender of their error and delete the email and any attachments without delay.
"""

In [9]:
# loading Company and emails

In [10]:
country = "Saudi Arabia"
files = os.listdir("../results/emails/{}".format(country))
len(files)

22

In [11]:
# Saving to inbox
class Mail:
    def __init__(self):
        # considering the same user and pass for smtp an imap
        self.mail_user = ''
        self.mail_pass = ''
        self.mail_host = 'smtp.hostinger.com'


    def send_email(self, to, subject, body, path, attach, attach2):
        message = MIMEMultipart()
        
#         message["From"] = self.mail_user
        message["From"] = formataddr(('Dr Macs Pharma', 'yesh@drmacspharma.com'))
        message["To"] = to
        message["Subject"] = subject
        message.attach(MIMEText(body, "plain"))
        
#         html = '<html><body><p>Here a just an html content!</p></body></html>'
#         part2 = MIMEText(html, 'html')
#         message.attach(part2)
        
        # attachment 1
        with open(path + attach, "rb") as attachment:
            part = MIMEBase("application", "octet-stream")
            part.set_payload(attachment.read())
        encoders.encode_base64(part)

        part.add_header(
            "Content-Disposition",
            "attachment; filename= \"" + attach + "\"",
        )
        message.attach(part)
#         print("1 attachment done.")

        
        # attachment 2
        with open(path + attach2, "rb") as attachment:
            part = MIMEBase("application", "octet-stream")
            part.set_payload(attachment.read())
        encoders.encode_base64(part)

        part.add_header(
            "Content-Disposition",
            "attachment; filename= \"" + attach2 + "\"",
        )
        message.attach(part)
        
#         print("2 attachment done.")
        
        text = message.as_string()
        
        context = ssl.create_default_context()
        with smtplib.SMTP_SSL(self.mail_host, context=context) as server:
            result = server.login(self.mail_user, self.mail_pass)
            server.sendmail(self.mail_user, to, text)

        imap = imaplib.IMAP4_SSL('imap.hostinger.com',993)
        imap.login(self.mail_user, self.mail_pass)
        imap.append('INBOX.Sent', '\\Seen', imaplib.Time2Internaldate(time.time()), text.encode('utf8'))
        imap.logout()


In [12]:
m = Mail()

In [13]:
for ind, file in enumerate(files):
    print("Index ::",ind)
    print("Started for file ::",file)
    
    emails_df = pd.read_csv("../results/emails/{}/{}".format(country,file))
    rows = emails_df.to_dict("records")
    
    for r in tqdm.tqdm(rows):
        if type(r['emails'])!=float:
            name = r['name']
            message = template.replace("[Company Name]",name)
            emails = r['emails'].split(";")
            for e in emails:
                m.send_email(e, subject, message, '', '../data/Dr Macs Pharma Brochure Arabic.pdf', '../data/Dr Macs Pharma Brochure.pdf')
    

Index :: 0
Started for file :: Abha_companies.csv


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:24<00:00,  4.02s/it]


Index :: 1
Started for file :: Abqaiq_companies.csv


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:33<00:00,  1.67s/it]


Index :: 2
Started for file :: Ad Dammam_companies.csv


...

100%|██████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<?, ?it/s]


Index :: 20
Started for file :: Al Mubarraz_companies.csv


100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]


Index :: 21
Started for file :: Al Qatif_companies.csv


100%|██████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<?, ?it/s]


## LLM based hyper-personalized email

In [3]:
client_website = ""
prospect_website = ""

In [4]:
# Step 1: Extract content from client website
# Step 2: Extract products and services from client website
# Step 3: Extract content from prospect_website
# Step 4: Summarize prospect_website content
# Step 5: Create personalized email to prospect using LLM model


#### LLM modeo

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

# model 
model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)  

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
## Generating a code block
prompt = """
Role: You are a marketing executive for the {client} company. You need to create a personalized marketing email to prospect company. 
Here are the services and products {client} company offers. 
Here is the information about the {prospect} company. 
{}

Now creat a personalized email, so that our services and products perfectly match with {prospect} company. 
Subject:
Email body:

"""


inputs = tokenizer(prompt, return_tensors="pt")

output = tokenizer.decode(model.generate(
    inputs["input_ids"],
    max_new_tokens=100)[0], skip_special_tokens=True)

print("Generated output")
print(output)